# Divide input dataset into NxN pixel tiles

In [1]:
import pathlib
import cv2
import numpy as np
import tqdm
import shutil
import random

## Data folders

In [2]:
data_path = pathlib.Path('../../data/')
inria_path = data_path / 'INRIA'
output_path = inria_path / 'output'
tmp_path = output_path / 'tmp'

train_path_tmp = tmp_path / 'train'
test_path_tmp = tmp_path / 'test'

train_path = output_path / 'train'
train_images_path = train_path / 'image'
train_images_path.mkdir(parents=True, exist_ok=True)
train_labels_path = train_path / 'label'
train_labels_path.mkdir(parents=True, exist_ok=True)

val_path = output_path / 'val'
val_images_path = val_path / 'image'
val_images_path.mkdir(parents=True, exist_ok=True)
val_labels_path = val_path / 'label'
val_labels_path.mkdir(parents=True, exist_ok=True)

test_path = output_path / 'test'
test_path.mkdir(parents=True, exist_ok=True)

## Dividing training data
Output will be TILE_HxTILE_W .png images

In [3]:
# Divide training data into train and val
paths = list(train_path_tmp.glob('images/*.tif'))
random.shuffle(paths)

n_train = len(paths) * 0.8

train_paths = paths[:int(n_train)]
val_paths = paths[int(n_train):]

In [4]:
# Set to True to remove images without rooftops from the dataset
SKIP_IMG_WITHOUT_ROOFTOPS = True

# Set to True to save the label tiles also in a 255 range (easier to visualize)
SAVE_VIZ = False

TILE_H = 512
TILE_W = TILE_H

print('Dividing files into tiles...')
pbar = tqdm.tqdm(train_paths)
for file in pbar:
    pbar.set_description(file.name)
    
    img = cv2.imread(str(file))
    label = cv2.imread(str(train_path_tmp / 'gt' / file.name))
    label = (label // 255)
    
    # if there is no rooftop in the image, skip it
    if SKIP_IMG_WITHOUT_ROOFTOPS and not label.any():
        break
    
    height, width, dim = img.shape
    
    for h in range(0, height, TILE_H):
        for w in range(0, width, TILE_W):
            img_tile = img[h:h+TILE_H, w:w+TILE_W]
            img_tile = np.pad(img_tile, ((0, TILE_H - img_tile.shape[0]), (0, TILE_W - img_tile.shape[1]), (0, 0)),
                   mode='constant', constant_values=0)
            
            label_tile = label[h:h+TILE_H, w:w+TILE_W]
            label_tile = np.pad(label_tile, ((0, TILE_H - label_tile.shape[0]), (0, TILE_W - label_tile.shape[1]), (0, 0)),
                   mode='constant', constant_values=0)
            
            # if there is no rooftop in the tile, skip it
            if SKIP_IMG_WITHOUT_ROOFTOPS and label_tile.any():
                cv2.imwrite(str(train_images_path / ('{0}_{1}_{2}.png').format(file.stem, h, w)), img_tile)
                cv2.imwrite(str(train_labels_path / ('{0}_{1}_{2}.png').format(file.stem, h, w)), label_tile)
                if SAVE_VIZ:
                    cv2.imwrite(str(train_labels_path / ('{0}_{1}_{2}_viz.png').format(file.stem, h, w)), label_tile*255)

print('Finished!')

Dividing files into tiles...


tyrol-w34.tif: 100%|██████████████████████████████████████████████████████████████| 144/144 [04:34<00:00,  1.91s/it]

Finished!


In [5]:
# Set to True to remove images without rooftops from the dataset
SKIP_IMG_WITHOUT_ROOFTOPS = True

# Set to True to save the label tiles also in a 255 range (easier to visualize)
SAVE_VIZ = False

TILE_H = 512
TILE_W = TILE_H

print('Dividing files into tiles...')
pbar = tqdm.tqdm(val_paths)
for file in pbar:
    pbar.set_description(file.name)
    
    img = cv2.imread(str(file))
    label = cv2.imread(str(train_path_tmp / 'gt' / file.name))
    label = (label // 255)
    
    # if there is no rooftop in the image, skip it
    if SKIP_IMG_WITHOUT_ROOFTOPS and not label.any():
        break
    
    height, width, dim = img.shape
    
    for h in range(0, height, TILE_H):
        for w in range(0, width, TILE_W):
            img_tile = img[h:h+TILE_H, w:w+TILE_W]
            img_tile = np.pad(img_tile, ((0, TILE_H - img_tile.shape[0]), (0, TILE_W - img_tile.shape[1]), (0, 0)),
                   mode='constant', constant_values=0)
            
            label_tile = label[h:h+TILE_H, w:w+TILE_W]
            label_tile = np.pad(label_tile, ((0, TILE_H - label_tile.shape[0]), (0, TILE_W - label_tile.shape[1]), (0, 0)),
                   mode='constant', constant_values=0)
            
            # if there is no rooftop in the tile, skip it
            if SKIP_IMG_WITHOUT_ROOFTOPS and label_tile.any():
                cv2.imwrite(str(val_images_path / ('{0}_{1}_{2}.png').format(file.stem, h, w)), img_tile)
                cv2.imwrite(str(val_labels_path / ('{0}_{1}_{2}.png').format(file.stem, h, w)), label_tile)
                if SAVE_VIZ:
                    cv2.imwrite(str(val_labels_path / ('{0}_{1}_{2}_viz.png').format(file.stem, h, w)), label_tile*255)

print('Finished!')

Dividing files into tiles...


kitsap23.tif: 100%|█████████████████████████████████████████████████████████████████| 36/36 [01:05<00:00,  1.83s/it]

Finished!


## Dividing test data
Output will be TILE_HxTILE_W .png images

In [6]:
print('Dividing files into tiles...')
pbar = tqdm.tqdm(test_path_tmp.glob('images/*.tif'))
for file in pbar:
    pbar.set_description(file.name)
    
    img = cv2.imread(str(file))
    
    height, width, dim = img.shape
    
    for h in range(0, height, TILE_H):
        for w in range(0, width, TILE_W):
            img_tile = img[h:h+TILE_H, w:w+TILE_W]
            img_tile = np.pad(img_tile, ((0, TILE_H - img_tile.shape[0]), (0, TILE_W - img_tile.shape[1]), (0, 0)),
                   mode='constant', constant_values=0)
            
            cv2.imwrite(str(test_path / ('{0}_{1}_{2}.png').format(file.stem, h, w)), img_tile)
                
print('Finished!')

Dividing files into tiles...


sfo23.tif: : 180it [04:36,  1.54s/it]        

Finished!


## Removing tmp folder

In [7]:
print('Removing tmp folder...')
shutil.rmtree(tmp_path)
print('Finished!')

Removing tmp folder...
Finished!


## Final structure

```
data
|__INRIA
   |__input
   |  |  aerialimagelabeling.7z.001
   |  |  ...
   |  |  aerialimagelabeling.7z.005
   |
   |__output
      |__train
      |  |__image
      |     |  <ORIGINAL_FILENAME>_<INIT_CROP_H>_<INIT_CROP_W>.png
      |     |  ...
      |
      |  |__label
      |     |  <ORIGINAL_FILENAME>_<INIT_CROP_H>_<INIT_CROP_W>.png
      |     |  ...
      |
      |__val
      |  |__image
      |     |  <ORIGINAL_FILENAME>_<INIT_CROP_H>_<INIT_CROP_W>.png
      |     |  ...
      |
      |  |__label
      |     |  <ORIGINAL_FILENAME>_<INIT_CROP_H>_<INIT_CROP_W>.png
      |     |  ...
      |
      |__test
         |  <ORIGINAL_FILENAME>_<INIT_CROP_H>_<INIT_CROP_W>.png
         |  ...
```